# Autograd 

It is central package for all neural networks.

The autograd package provides automatic differentiation for all operations on Tensors.


In [1]:
import torch

Creat a tensor and set required_grad = True to track it's computation.

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do a tensor operation.

In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y was created using an operation on requires_grad=True tensor. So, it has grad_fn.

In [4]:
print(y.grad_fn)

Do more operation on y.

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


.requires\_grad\_() changes existing requires_grad of a tensor. By default requires_grad is False at declaring time.

In [6]:
a = torch.ones(2, 2)
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


## Gradients.

Let's backprop now. .backward() calculates differentiation with respect to requires_grad=True tensor.

out is a scalar so we don't pass any argument it treats it like out.backward(x)

In [7]:
out.backward()

We can access value of gradient by .grad attribute.

In [8]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You see 4x4 matrix. Because, x is 4x4.

autograd calculates gradient by vector-Jacobian product and chain rule.

You can stop autograd tracking either by wrapping code in with torch.no_grad() this will help in evaluating the model.

In [9]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False


Or by using .detach() to get new tensor with same content with no grad.

In [10]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(x).all())

True
False
tensor(True)


## Custom Autograd.

We can make our own autograd by subclassing autograd.Function.

In [11]:
class MyRelu(torch.autograd.Function):

    # Forwars pass.
    @staticmethod
    def forward(ctx, input):

        # ctx is a context object used to stash information of forward pass.
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    # Backward pass.
    @staticmethod
    def backward(ctx, grad_output):

        # We recieve gradient of loss w.r.t. output.
        # And return gradient of loss w.r.t. input.

        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0

        return grad_input


In [12]:
# To apply our custom function, we use function.apply method.
relu = MyRelu.apply

For more: https://pytorch.org/docs/stable/autograd.html